In [1]:
import pandas as pd
import numpy as np
import random
import tqdm
from project_lib import *

In [2]:
DATE_OF_SIMULATION = '2021-03-28'
SIGMA_SL = -1
SIGMA_TP = 1.5
MAX_TRADE_DURATION = 63
PTF_VOL_TGT = 0.19
AUM_LOST_SL = 0.021

In [3]:
df = pd.read_csv('dataset_tmc.csv', parse_dates = ['Dates'], date_format = '%d/%m/%y')
df.columns = [i.lower() for i in df.columns]
df.columns = [i.replace(' ','_') for i in df.columns]
df = df.set_index('dates')

# Check NaNs
if df.isna().sum().sum() != 0:
    print('check NaNs in data')

In [4]:
mc_opt_n_trades(df,
                incipit_date = DATE_OF_SIMULATION,
                sigma_sl = SIGMA_SL,
                sigma_tp = SIGMA_TP,
                max_trade_duration = MAX_TRADE_DURATION,
                ptf_vol_tgt = PTF_VOL_TGT,
                aum_lost_sl = AUM_LOST_SL,
                iterations=100)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:26<00:00,  3.74it/s]

Optimal Number of Trades: 46 ± 11 [with avg. portfolio volatility of 19.37%]


In [5]:
mc_fcast_ptf_volatility(df,
                incipit_date = DATE_OF_SIMULATION,
                sigma_sl = SIGMA_SL,
                sigma_tp = SIGMA_TP,
                max_trade_duration = MAX_TRADE_DURATION,
                number_of_trades = 46,
                aum_lost_sl = AUM_LOST_SL,
                iterations=100)

100%|██████████| 100/100 [00:26<00:00,  3.82it/s]

Estimated Portfolio Volatility: 19.33% ± 2.54%


In [4]:
mc_perc_aum_lost(df,
            incipit_date= DATE_OF_SIMULATION,
            sigma_sl = SIGMA_SL,
            sigma_tp = SIGMA_TP,
            max_trade_duration = MAX_TRADE_DURATION,
            ptf_vol_tgt = PTF_VOL_TGT,
            number_of_trades = 46,
            iterations=4)

  0%|          | 0/4 [00:00<?, ?it/s]

100%|██████████| 4/4 [00:18<00:00,  4.60s/it]

Estimated %AUM lost per trade: 2.0% ± 0.16%
